In [ ]:
!pip install openai
!pip install aiofiles aiohttp


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.1/599.1 kB 3.6 MB/s eta 0:00:00


In [ ]:
import openai
import base64
import os

# STEP 1: Set your OpenAI API key
client = openai.OpenAI(api_key="nn")  # Replace with your key

# STEP 2: Encode image to base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# STEP 3: Image file path
image_path = "/Users/fatihwolf/Downloads/output (2)/row_6_teacher.png"
base64_image = encode_image(image_path)

# STEP 4: Send request to GPT-4 Vision
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": """You are a vision-language assistant. Your task is to describe an image using exactly six predefined categories. You MUST return your response in a **single line**, using the following key-value format:

main_objects: ..., main_object_attributes: ..., location: ..., action: ..., surroundings: ..., background: ...

Each category must be filled with detailed and specific visual content extracted from the image. Do NOT use vague terms like "stuff," "something," or "object." Do not include commentary, explanations, greetings, or extra text. Follow this structure **exactly** and use commas to separate phrases **within** each category (not between categories). Here's what each category must include:

- **main_objects**: The visually dominant object(s) in the image. Use nouns only (e.g., dog, man, car). Plural if needed.
- **main_object_attributes**: Descriptive attributes of the main objects. Include color, size, shape, texture, pose, clothing, breed, etc. Be specific and avoid general adjectives like "nice" or "good."
- **location**: The specific place or physical setting of the main objects. It can be indoor (e.g., kitchen, classroom) or outdoor (e.g., beach, forest). Be precise.
- **action**: The primary visible activity or behavior of the main object(s). Use verbs (e.g., eating, running, playing). If nothing is happening, say "none."
- **surroundings**: Notable objects, furniture, or elements near the main object(s). Focus on things in the foreground or immediate context.
- **background**: What is behind the main objects — it can include landscape elements (e.g., mountains, sky), walls, scenery, or distant elements.

Output Format Example (follow this EXACTLY, only change content):
main_objects: dog, girl; main_object_attributes: golden retriever, medium size, fluffy fur, barefoot child in red dress; location: grassy park; action: playing fetch; surroundings: ball, picnic basket, tree; background: blue sky with clouds, distant buildings
"""},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                    }
                }
            ]
        }
    ],
    max_tokens=300,
)

# STEP 5: Print the output
print("Generated Caption:\n")
print(response.choices[0].message.content)


Generated Caption:

main_objects: boy, soccer ball; main_object_attributes: red jersey, blue shorts, black shoes, medium size ball; location: soccer field; action: kicking; surroundings: grass, goalposts; background: bright sun, blue sky with clouds, trees


In [ ]:
import openai
import base64
import os

# STEP 1: Set your OpenAI API key
client = openai.OpenAI(api_key="")  # Replace with your key

# STEP 2: Encode image to base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# STEP 3: Image file path
image_path = "/Users/fatihwolf/Downloads/output (2)/row_6_student.png"
base64_image = encode_image(image_path)

# STEP 4: Send request to GPT-4 Vision
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": """You are a vision-language assistant. Your task is to describe an image using exactly six predefined categories. You MUST return your response in a **single line**, using the following key-value format:

main_objects: ..., main_object_attributes: ..., location: ..., action: ..., surroundings: ..., background: ...

Each category must be filled with detailed and specific visual content extracted from the image. Do NOT use vague terms like "stuff," "something," or "object." Do not include commentary, explanations, greetings, or extra text. Follow this structure **exactly** and use commas to separate phrases **within** each category (not between categories). Here's what each category must include:

- **main_objects**: The visually dominant object(s) in the image. Use nouns only (e.g., dog, man, car). Plural if needed.
- **main_object_attributes**: Descriptive attributes of the main objects. Include color, size, shape, texture, pose, clothing, breed, etc. Be specific and avoid general adjectives like "nice" or "good."
- **location**: The specific place or physical setting of the main objects. It can be indoor (e.g., kitchen, classroom) or outdoor (e.g., beach, forest). Be precise.
- **action**: The primary visible activity or behavior of the main object(s). Use verbs (e.g., eating, running, playing). If nothing is happening, say "none."
- **surroundings**: Notable objects, furniture, or elements near the main object(s). Focus on things in the foreground or immediate context.
- **background**: What is behind the main objects — it can include landscape elements (e.g., mountains, sky), walls, scenery, or distant elements.

Output Format Example (follow this EXACTLY, only change content):
main_objects: dog, girl; main_object_attributes: golden retriever, medium size, fluffy fur, barefoot child in red dress; location: grassy park; action: playing fetch; surroundings: ball, picnic basket, tree; background: blue sky with clouds, distant buildings
"""},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                    }
                }
            ]
        }
    ],
    max_tokens=300,
)

# STEP 5: Print the output
print("Generated Caption:\n")
print(response.choices[0].message.content)


Generated Caption:

main_objects: boy, soccer ball; main_object_attributes: young, blonde hair, blue jersey, black shorts; location: soccer field; action: dribbling; surroundings: goalposts, other soccer balls; background: blurred trees, house, blue sky


In [ ]:
import openai
import base64
import os
import re
import asyncio
import aiofiles
import pandas as pd
from aiohttp import ClientSession, ClientTimeout, TCPConnector

# Set your OpenAI API key
API_KEY = "API KEY HERE"
client = openai.AsyncOpenAI(api_key=API_KEY)

FOLDER_PATH = "FOLDER NAME HERE"
MAX_CONCURRENT_REQUESTS = 10  # Limit concurrent requests to prevent errors

prompt_text = """You are a vision-language assistant. Your task is to describe an image using exactly six predefined categories. You MUST return your response in a **single line**, using the following key-value format:

main_objects: ..., main_object_attributes: ..., location: ..., action: ..., surroundings: ..., background: ...

Each category must be filled with detailed and specific visual content extracted from the image. Do NOT use vague terms like "stuff," "something," or "object." Do not include commentary, explanations, greetings, or extra text. Follow this structure **exactly** and use commas to separate phrases **within** each category (not between categories). Here's what each category must include:

- **main_objects**: The visually dominant object(s) in the image. Use nouns only (e.g., dog, man, car). Plural if needed.
- **main_object_attributes**: Descriptive attributes of the main objects. Include color, size, shape, texture, pose, clothing, breed, etc. Be specific and avoid general adjectives like "nice" or "good."
- **location**: The specific place or physical setting of the main objects. It can be indoor (e.g., kitchen, classroom) or outdoor (e.g., beach, forest). Be precise.
- **action**: The primary visible activity or behavior of the main object(s). Use verbs (e.g., eating, running, playing). If nothing is happening, say "none."
- **surroundings**: Notable objects, furniture, or elements near the main object(s). Focus on things in the foreground or immediate context.
- **background**: What is behind the main objects — it can include landscape elements (e.g., mountains, sky), walls, scenery, or distant elements.

Output Format Example (follow this EXACTLY, only change content):
main_objects: dog, girl; main_object_attributes: golden retriever, medium size, fluffy fur, barefoot child in red dress; location: grassy park; action: playing fetch; surroundings: ball, picnic basket, tree; background: blue sky with clouds, distant buildings
"""

# Load images asynchronously
async def encode_image_async(image_path):
    async with aiofiles.open(image_path, "rb") as image_file:
        return base64.b64encode(await image_file.read()).decode("utf-8")

# Function to send a request to OpenAI API
async def process_image(image_path, session, semaphore):
    async with semaphore:
        image_base64 = await encode_image_async(image_path)
        response = await client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": [
                {"type": "text", "text": prompt_text},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_base64}"}}
            ]}],
            max_tokens=300,
        )
        return response.choices[0].message.content

# Process all images in parallel with controlled concurrency
async def process_all_images():
    image_files = sorted(f for f in os.listdir(FOLDER_PATH) if f.endswith(('.png', '.jpg', '.jpeg')))
    
    # Match teacher & student images
    image_pairs = {}
    for filename in image_files:
        match = re.match(r"row_(\d+)_(teacher|student)\.(png|jpg|jpeg)", filename)
        if match:
            row_number, role, _ = match.groups()
            if row_number not in image_pairs:
                image_pairs[row_number] = {}
            image_pairs[row_number][role] = filename

    results = []
    semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)

    async with ClientSession(timeout=ClientTimeout(total=60), connector=TCPConnector(limit=MAX_CONCURRENT_REQUESTS)) as session:
        tasks = []
        
        for row, images in sorted(image_pairs.items(), key=lambda x: int(x[0])):
            teacher_img = images.get("teacher")
            student_img = images.get("student")
            if teacher_img and student_img:
                teacher_path = os.path.join(FOLDER_PATH, teacher_img)
                student_path = os.path.join(FOLDER_PATH, student_img)
                
                tasks.append(process_image(teacher_path, session, semaphore))
                tasks.append(process_image(student_path, session, semaphore))

        responses = await asyncio.gather(*tasks)
        
        for i, (row, images) in enumerate(sorted(image_pairs.items(), key=lambda x: int(x[0]))):
            results.append({
                "Row": row,
                "Teacher Caption": responses[i * 2],
                "Student Caption": responses[i * 2 + 1]
            })

    # Save results
    df = pd.DataFrame(results)
    df.to_csv("results.csv", index=False)

    print("✅ Processing complete! Results saved to 'results.csv'.")

# Run the async function
asyncio.run(process_all_images())


In [ ]:
## FOR BIGGER DATA -- THERE IS A LIMIT PROBLEM

import openai
import base64
import os
import re
import asyncio
import aiofiles
import pandas as pd
from aiohttp import ClientSession, ClientTimeout, TCPConnector
import random
from openai import RateLimitError, APIError

# Set your OpenAI API key
API_KEY = "YOUR_API_KEY_HERE"
client = openai.AsyncOpenAI(api_key=API_KEY)

FOLDER_PATH = "YOUR_IMAGES_FOLDER"
MAX_CONCURRENT_REQUESTS = 10  # Limit concurrent requests to prevent errors

prompt_text = """You are a vision-language assistant. Your task is to describe an image using exactly six predefined categories. You MUST return your response in a **single line**, using the following key-value format:

main_objects: ..., main_object_attributes: ..., location: ..., action: ..., surroundings: ..., background: ...

Each category must be filled with detailed and specific visual content extracted from the image. Do NOT use vague terms like "stuff," "something," or "object." Do not include commentary, explanations, greetings, or extra text. Follow this structure **exactly** and use commas to separate phrases **within** each category (not between categories). Here's what each category must include:

- **main_objects**: The visually dominant object(s) in the image. Use nouns only (e.g., dog, man, car). Plural if needed. Do not include labels like "cartoon cat"—only the object itself.
- **main_object_attributes**: Descriptive attributes of the main objects. Include color, size, shape, texture, pose, clothing, breed, etc. Be specific and avoid general adjectives like "nice" or "good."
- **location**: The specific place or physical setting of the main objects. It can be indoor (e.g., kitchen, classroom) or outdoor (e.g., beach, forest). Be precise.
- **action**: The primary visible activity or behavior of the main object(s). Use verbs (e.g., eating, running, playing). If nothing is happening, say "none."
- **surroundings**: Notable objects, furniture, or elements near the main object(s). Focus on things in the foreground or immediate context.
- **background**: What is behind the main objects — it can include landscape elements (e.g., mountains, sky), walls, scenery, or distant elements.

Output Format Example (follow this EXACTLY, only change content):
main_objects: dog, girl; main_object_attributes: golden retriever, medium size, fluffy fur, barefoot child in red dress; location: grassy park; action: playing fetch; surroundings: ball, picnic basket, tree; background: blue sky with clouds, distant buildings
"""

# Load images asynchronously
async def encode_image_async(image_path):
    async with aiofiles.open(image_path, "rb") as image_file:
        return base64.b64encode(await image_file.read()).decode("utf-8")

async def process_image(image_path, session, semaphore, max_retries=5):
    async with semaphore:
        image_base64 = await encode_image_async(image_path)

        for attempt in range(max_retries):
            try:
                response = await client.chat.completions.create(
                    model="gpt-4o",
                    messages=[{"role": "user", "content": [
                        {"type": "text", "text": prompt_text},
                        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_base64}"}}
                    ]}],
                    max_tokens=300,
                )
                return response.choices[0].message.content

            except RateLimitError:
                print("⏳ Rate limit hit. Waiting 60 seconds before retrying...")
                await asyncio.sleep(60)  # Hard wait instead of exponential backoff

            except APIError as e:
                wait_time = 2 ** attempt + random.uniform(0, 1)
                print(f"⚠️ API error occurred: {e}. Retrying in {wait_time:.2f} seconds...")
                await asyncio.sleep(wait_time)

        print(f"❌ Failed to process image: {image_path} after {max_retries} attempts.")
        return "ERROR"

# Process all images in parallel with controlled concurrency
async def process_all_images():
    image_files = sorted(f for f in os.listdir(FOLDER_PATH) if f.endswith(('.png', '.jpg', '.jpeg')))

    # Match teacher & student images
    image_pairs = {}
    for filename in image_files:
        match = re.match(r"row_(\d+)_(teacher|student)\.(png|jpg|jpeg)", filename)
        if match:
            row_number, role, _ = match.groups()
            if row_number not in image_pairs:
                image_pairs[row_number] = {}
            image_pairs[row_number][role] = filename

    results = []
    semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)

    async with ClientSession(timeout=ClientTimeout(total=60), connector=TCPConnector(limit=MAX_CONCURRENT_REQUESTS)) as session:
        tasks = []
        row_list = sorted(image_pairs.items(), key=lambda x: int(x[0]))

        for row, images in row_list:
            teacher_img = images.get("teacher")
            student_img = images.get("student")
            if teacher_img and student_img:
                teacher_path = os.path.join(FOLDER_PATH, teacher_img)
                student_path = os.path.join(FOLDER_PATH, student_img)

                tasks.append(process_image(teacher_path, session, semaphore))
                tasks.append(process_image(student_path, session, semaphore))

        responses = await asyncio.gather(*tasks)

        for i, (row, _) in enumerate(row_list):
            results.append({
                "Row": row,
                "Teacher Caption": responses[i * 2],
                "Student Caption": responses[i * 2 + 1]
            })

    # Save results
    df = pd.DataFrame(results)
    df.to_csv("captions.csv", index=False)

    print("✅ Processing complete! Results saved to 'captions.csv'.")

# Allow running in Jupyter/Colab without asyncio.run error
import nest_asyncio
nest_asyncio.apply()

await process_all_images()


In [ ]:
import openai
import base64
import os
import re
import asyncio
import aiofiles
import pandas as pd
from aiohttp import ClientSession, ClientTimeout, TCPConnector
import random
from openai import RateLimitError, APIError

# Set your OpenAI API key
API_KEY = "YOUR_API_KEY_HERE"
client = openai.AsyncOpenAI(api_key=API_KEY)

FOLDER_PATH = "IMAGES_FOLDER"
MAX_CONCURRENT_REQUESTS = 10  # Limit concurrent requests to prevent errors

# Set the row range you want to process
MIN_ROW = 0  # Inclusive start row
MAX_ROW = 20  # Inclusive end row

prompt_text = """You are a vision-language assistant. Your task is to describe an image using exactly six predefined categories. You MUST return your response in a **single line**, using the following key-value format:

main_objects: ..., main_object_attributes: ..., location: ..., action: ..., surroundings: ..., background: ...

Each category must be filled with detailed and specific visual content extracted from the image. Do NOT use vague terms like "stuff," "something," or "object." Do not include commentary, explanations, greetings, or extra text. Follow this structure **exactly** and use commas to separate phrases **within** each category (not between categories). Here's what each category must include:

- **main_objects**: The visually dominant object(s) in the image. Use nouns only (e.g., dog, man, car). Plural if needed. Do not include labels like "cartoon cat"—only the object itself.
- **main_object_attributes**: Descriptive attributes of the main objects. Include color, size, shape, texture, pose, clothing, breed, etc. Be specific and avoid general adjectives like "nice" or "good."
- **location**: The specific place or physical setting of the main objects. It can be indoor (e.g., kitchen, classroom) or outdoor (e.g., beach, forest). Be precise.
- **action**: The primary visible activity or behavior of the main object(s). Use verbs (e.g., eating, running, playing). If nothing is happening, say "none."
- **surroundings**: Notable objects, furniture, or elements near the main object(s). Focus on things in the foreground or immediate context.
- **background**: What is behind the main objects — it can include landscape elements (e.g., mountains, sky), walls, scenery, or distant elements.

Output Format Example (follow this EXACTLY, only change content):
main_objects: dog, girl; main_object_attributes: golden retriever, medium size, fluffy fur, barefoot child in red dress; location: grassy park; action: playing fetch; surroundings: ball, picnic basket, tree; background: blue sky with clouds, distant buildings
"""

# Load images asynchronously
async def encode_image_async(image_path):
    async with aiofiles.open(image_path, "rb") as image_file:
        return base64.b64encode(await image_file.read()).decode("utf-8")

async def process_image(image_path, session, semaphore, max_retries=5):
    async with semaphore:
        image_base64 = await encode_image_async(image_path)

        for attempt in range(max_retries):
            try:
                response = await client.chat.completions.create(
                    model="gpt-4o",
                    messages=[{"role": "user", "content": [
                        {"type": "text", "text": prompt_text},
                        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_base64}"}}
                    ]}],
                    max_tokens=300,
                )
                return response.choices[0].message.content

            except RateLimitError as e:
                wait_time = 2 ** attempt + random.uniform(0, 1)
                print(f"⚠️ Rate limit hit. Retrying in {wait_time:.2f} seconds...")
                await asyncio.sleep(wait_time)

            except APIError as e:
                wait_time = 2 ** attempt + random.uniform(0, 1)
                print(f"⚠️ API error occurred: {e}. Retrying in {wait_time:.2f} seconds...")
                await asyncio.sleep(wait_time)

        print(f"❌ Failed to process image: {image_path} after {max_retries} attempts.")
        return "ERROR"

# Process all images in parallel with controlled concurrency
async def process_all_images():
    image_files = sorted(f for f in os.listdir(FOLDER_PATH) if f.endswith(('.png', '.jpg', '.jpeg')))

    # Match teacher & student images
    image_pairs = {}
    for filename in image_files:
        match = re.match(r"row_(\d+)_(teacher|student)\.(png|jpg|jpeg)", filename)
        if match:
            row_number, role, _ = match.groups()
            if row_number not in image_pairs:
                image_pairs[row_number] = {}
            image_pairs[row_number][role] = filename

    results = []
    semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)

    async with ClientSession(timeout=ClientTimeout(total=60), connector=TCPConnector(limit=MAX_CONCURRENT_REQUESTS)) as session:
        tasks = []

        for row, images in sorted(image_pairs.items(), key=lambda x: int(x[0])):
            row_num = int(row)
            if not (MIN_ROW <= row_num <= MAX_ROW):
                continue

            teacher_img = images.get("teacher")
            student_img = images.get("student")
            if teacher_img and student_img:
                teacher_path = os.path.join(FOLDER_PATH, teacher_img)
                student_path = os.path.join(FOLDER_PATH, student_img)

                tasks.append(process_image(teacher_path, session, semaphore))
                tasks.append(process_image(student_path, session, semaphore))

        responses = await asyncio.gather(*tasks)

        filtered_pairs = [(row, images) for row, images in sorted(image_pairs.items(), key=lambda x: int(x[0])) if MIN_ROW <= int(row) <= MAX_ROW]
        for i, (row, images) in enumerate(filtered_pairs):
            results.append({
                "Row": row,
                "Teacher Caption": responses[i * 2],
                "Student Caption": responses[i * 2 + 1]
            })

    # Save results
    df = pd.DataFrame(results)
    df.to_csv("captions.csv", index=False)

    print("✅ Processing complete! Results saved to 'captions.csv'.")

# Run the async function
asyncio.run(process_all_images())
